In [2]:
""" 
THIS IS THE LEFTOVER CODE FROM ATTEMPTING TO USE HMMs

HMMs are memory-less and thus required linear states of lenght = pos_motif
in order to put the motif exactly where stated. This quickly led to very large 
HMMs that will grow exponentially in size with added motifs. 

This is here to compare the time of execution between sampling via this 
HMM technique or the simple for loop numpy.random idea. 
"""

' \nTHIS IS THE LEFTOVER CODE FROM ATTEMPTING TO USE HMMs\n\nHMMs are memory-less and thus required linear states of lenght = pos_motif\nin order to put the motif exactly where stated. This quickly led to very large \nHMMs that will grow exponentially in size with added motifs. \n\nThis is here to compare the time of execution between sampling via this \nHMM technique or the simple for loop numpy.random idea. \n'

In [3]:
import numpy
import pomegranate as pm
import matplotlib.pyplot as plt

In [4]:
def simuldna_background(gc_frac):
    """
    Generates an HMM to output background sequences
    
    Args:
        gc_frac: float between 0 and 1 representing the fracton of G's and C's in the 
            background (non-motif) dna sequence
    
    Returns:
        A pomegranate.HiddenMarkovModel to be .sample() for background sequences
    """
    backg_model = pm.HiddenMarkovModel('simuldna_background')
    c_frac = gc_frac/2 
    bg = pm.State(pm.DiscreteDistribution({'A' : (1/2-c_frac), 'C' : c_frac, 'G' : c_frac, 'T' : (1/2-c_frac)}), name="bg")
    backg_model.add_state(bg)
    backg_model.add_transition(backg_model.start, bg, 1)
    backg_model.add_transition(bg, bg, 1)    
    backg_model.bake()
    return backg_model
    

In [5]:
def simuldna_one(pwm_motif, gc_frac, when_motif, **kwargs): # (int, ndarray([A], [C], [G], [T]), float, float)
    """ Generates an HMM with a motif state to be transitioned to by specified chance (when_motif)
    
    This is not used in the comparison of time of execution. And is still here because I have become
    to code I write.
    
    """
    model = pm.HiddenMarkovModel('simuldna_one')
    c_frac = gc_frac/2           # half the input fraction goes for C's and G's, 1-c_frac for A's and T's
    # make catch cases for if the frac is >1 and if pwm_motif isn't an array and if len isn't an int
    # and for if when_motif > 1
    
    len_motif = pwm_motif.shape[1] # the length of the motif
    
    # background distributions
    bg = pm.State(pm.DiscreteDistribution({'A' : (1/2-c_frac), 'C' : c_frac, 'G' : c_frac, 'T' : (1/2-c_frac)}), name="bg")
    bg_no = pm.State(pm.DiscreteDistribution({'A' : (1/2-c_frac), 'C' : c_frac, 'G' : c_frac, 'T' : (1/2-c_frac)}), name="bg_no")
    
    models = []
    
    for i in range(len_motif): # making motif states from given pwm matrix and adding to model
        models.append(pm.State(pm.DiscreteDistribution({'A' : pwm_motif[0][i], 'C' : pwm_motif[1][i], 'G' : pwm_motif[2][i], 'T' : pwm_motif[3][i]}), name="m"+str(i+1)))
        model.add_state(pm.State(pm.DiscreteDistribution({'A' : pwm_motif[0][i], 'C' : pwm_motif[1][i], 'G' : pwm_motif[2][i], 'T' : pwm_motif[3][i]}), name="m"+str(i+1)))
        
    model.add_states(bg, bg_no) # adding background states
    
    # adding transition probabilities
    model.add_transition(model.start, bg, (1-when_motif))
    model.add_transition(model.start, models[0], when_motif) # motif can start on X1 position
    model.add_transition(bg, bg, (1-when_motif))
    model.add_transition(bg, models[0], when_motif)
    for i in range(len_motif-1):
        model.add_transition(models[i], models[i+1], 1)
    model.add_transition(models[len_motif-1], bg_no, 1)
    model.add_transition(bg_no, bg_no, 1)
    # no end transition probability because length is specified
    
    model.bake() # this makes the model usable
    
    
    return model

In [6]:
def simuldna_spec(length, pwm_motif, gc_frac, where_motif, num_samples): 
    """Outputs list of samples with a singular motif at specified index.
    
    Args: 
        see new_simuldna_dist
    """
    
    # TODO: if gc_frac > 1: raise("needs to be a fraction of G's and C's in the background DNA sequence")
    # TODO: if where_motif < 1: raise("needs to be the index of location of protein motif")
    # TODO: if length < where_motif: raise "motif can't be placed outside the total length of DNA")
    

    
    # generate models BEFORE sampling tons of times
    bg_model = simuldna_background(gc_frac)
    # sequence where motif is at the start to extend the background sequence
    motif_model = simuldna_one(pwm_motif, gc_frac, 1) 
    
    
    bg_seq = []
    total = []
    for i in range(num_samples): 
        # TODO choose where_motif from a distribution perhaps (?)
        
        len_motif_seq = length-where_motif+1 
        bg_seq = bg_model.sample(length=where_motif-1)
        motif_seq = motif_model.sample(length=len_motif_seq)
        total.append(numpy.concatenate((bg_seq, motif_seq), axis=None))
    return total